# Metacognitive Prompting

LLMに人間の認知プロセスを模倣させることで精度を改善した手法．
- 論文:https://arxiv.org/abs/2308.05342

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/develop/nlp/llm_prompt_engineering/metacognitive_promting.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install openai

In [1]:
import os
from openai import OpenAI
client = OpenAI()

#os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

### 通常のプロンプト(Few-shot prompting)

In [2]:
prompt = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
"""

In [3]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
    )

print(response.choices[0].message.content)

これらの2つの質問はお互いの言い換えではありません。質問1は「世界で最も美しいビーチは何ですか？」という特定の尺度で美しさを求める一方で、質問2は単に「最も美しいビーチは何ですか？」という一般的な美しさを尋ねています。質問2は特定の範囲や尺度を示さず、意図する美しさの基準や制限が明確ではありません。したがって、これらは異なる意味と言えます。


### Metacognitive Prompting

In [4]:
prompt1 = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
Q:両方の質問に対するあなたの理解を明確にしてください。
A:
"""

In [6]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt1}]
    )

output1 = response.choices[0].message.content

print(response.choices[0].message.content)

質問1: 「世界で最も美しいビーチは何ですか？」
この質問は、世界中のビーチの中で最も美しいビーチが何かを尋ねています。

質問2:「最も美しいビーチは何ですか？」
この質問も、最も美しいビーチが何かを尋ねていますが、特定の地域や制約を設けずに尋ねています。

これら2つの質問は意味が似ていますが、質問2はより一般的で広範囲な尋ね方です。質問1は、美しいビーチに特化しており、世界中のビーチの中で最も美しいものが何かを尋ねています。質問2は、美しいビーチを求めており、特定の地域や制約を問わずに最も美しいビーチが何かを尋ねています。言い換えと言えるかどうかは、質問の意図や尋ね方のニュアンスによると思います。


In [8]:
prompt2 = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
Q:両方の質問に対するあなたの理解を明確にしてください。
A:{output1_}
Q:主題、文脈、意味内容に基づく類似性の予備的識別を行う。
A:
""".format(
    output1_=output1
    )

In [9]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt2}]
    )

output2 = response.choices[0].message.content

print(response.choices[0].message.content)

質問1と質問2の類似性は、質問の主題が「最も美しいビーチ」であることから始まります。両方の質問は、美しいビーチに関する情報を求めており、類似した意味内容を持っています。

ただし、少しニュアンスが異なります。質問1は「世界で最も美しいビーチは何ですか？」と述べており、世界中のビーチの中で最も美しいビーチを尋ねています。一方、質問2は「最も美しいビーチは何ですか？」と述べており、特定の地域や制約を設けずに最も美しいビーチを尋ねています。

質問1はより具体的で、特定の地域や制約を明示的に排除しています。質問2は一般的であり、地域や制約を問わずに美しいビーチを探しています。ただし、質問の主題と意図は類似しており、質問の文脈や目的によっては、質問1と質問2を言い換えとして扱うこともできます。


In [10]:
prompt3 = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
Q:両方の質問に対するあなたの理解を明確にしてください。
A:{output1_}
Q:主題、文脈、意味内容に基づく類似性の予備的識別を行う。
A:{output2_}
Q:予備的な分析を批判的に評価する。質問がパラフレーズであるという最初の評価に確信が持てない場合は、再評価を試みる。
A:
""".format(
    output1_=output1,
    output2_=output2
    )

In [11]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt3}]
    )

output3 = response.choices[0].message.content

print(response.choices[0].message.content)

質問1と質問2の間には類似性がありますが、完全なパラフレーズではないと言えます。質問1はより具体的で、世界中のビーチの中で最も美しいものを求めています。一方、質問2はより一般的で、特定の地域や制約を設けずに美しいビーチを求めています。

再評価してみると、質問1と質問2は似た意味内容を持っていると言えますが、質問2はより広範囲で一般的な尋ね方をしています。それに対して、質問1はより具体的な条件を設定しています。

したがって、質問1と質問2は部分的にパラフレーズと言えますが、完全な言い換えではありません。ニュアンスや言葉の遣い方に微妙な違いがありますので、注意が必要です。


In [13]:
prompt4 = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
Q:両方の質問に対するあなたの理解を明確にしてください。
A:{output1_}
Q:主題、文脈、意味内容に基づく類似性の予備的識別を行う。
A:{output2_}
Q:予備的な分析を批判的に評価する。質問がパラフレーズであるという最初の評価に確信が持てない場合は、再評価を試みる。
A:{output3_}
Q:質問がパラフレーズであるかどうかの最終的な判断を確認し、判断の根拠を示します。
A:
""".format(
    output1_=output1,
    output2_=output2,
    output3_=output3
    )

In [14]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt4}]
    )

output4 = response.choices[0].message.content

print(response.choices[0].message.content)

質問1と質問2は部分的にパラフレーズと言えますが、完全なパラフレーズではありません。

質問の主題や意図は類似しており、両方の質問は美しいビーチに関する情報を求めています。

質問1は「世界で最も美しいビーチは何ですか？」と述べており、特定の地域や制約を排除して世界中のビーチの中で最も美しいものを尋ねています。

一方、質問2は「最も美しいビーチは何ですか？」と述べており、特定の地域や制約を設けずに一般的な美しいビーチを尋ねています。

質問2はより広範囲で一般的な尋ね方をしており、質問1よりも具体的な条件がありません。

したがって、質問1と質問2は部分的にパラフレーズと言えますが、完全な言い換えではありません。


In [15]:
prompt5 = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
Q:両方の質問に対するあなたの理解を明確にしてください。
A:{output1_}
Q:主題、文脈、意味内容に基づく類似性の予備的識別を行う。
A:{output2_}
Q:予備的な分析を批判的に評価する。質問がパラフレーズであるという最初の評価に確信が持てない場合は、再評価を試みる。
A:{output3_}
Q:質問がパラフレーズであるかどうかの最終的な判断を確認し、判断の根拠を示します。
A:{output4_}
""".format(
    output1_=output1,
    output2_=output2,
    output3_=output3,
    output4_=output4
    )

In [16]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt5}]
    )

output5 = response.choices[0].message.content

print(response.choices[0].message.content)

質問の根拠は、質問の主題や意図が類似しており、美しいビーチに関する情報を求めている点です。しかし、質問1は特定の地域や制約を排除している一方、質問2は一般的な美しいビーチを尋ねています。これらの微妙な違いから、質問1と質問2は部分的にパラフレーズであり、完全な言い換えではないと判断します。


In [17]:
prompt6 = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
Q:両方の質問に対するあなたの理解を明確にしてください。
A:{output1_}
Q:主題、文脈、意味内容に基づく類似性の予備的識別を行う。
A:{output2_}
Q:予備的な分析を批判的に評価する。質問がパラフレーズであるという最初の評価に確信が持てない場合は、再評価を試みる。
A:{output3_}
Q:質問がパラフレーズであるかどうかの最終的な判断を確認し、判断の根拠を示します。
A:{output4_}
Q:自分の分析に対する確信度（0～100％）を評価し、その確信度の説明をする。
A:
""".format(
    output1_=output1,
    output2_=output2,
    output3_=output3,
    output4_=output4,
    output5_=output5
    )

In [18]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt6}]
    )

print(response.choices[0].message.content)

確信度: 80％

上記の分析は、質問1と質問2の類似性や違いについて適切に説明しています。主題や意図の類似性に基づいてパラフレーズであることを示しましたが、一部の微妙なニュアンスや特定の条件の有無による違いを指摘しました。ただし、文脈や目的によっては、質問1と質問2を言い換えとして扱うこともできます。
